#  Runnable Interface in LangChain
###  Understanding LangChains Modern Execution Engine

This notebook provides a structured, beginnerfriendly walkthrough of LangChains `Runnable` interface  a powerful and flexible way to build LLM-powered pipelines. Youll explore concepts, examples, and handson demos!


##  Learning Guide
In this notebook, you will learn:
- What the `Runnable` interface is and why it powers LangChains new architecture
- How different runnable types (Map, Sequence, Lambda, Branch) work
- How to build composable and scalable workflows
- How sync & async execution models improve performance

###  Why This Matters
`Runnable` is the *core execution engine* in LangChain, making chains more flexible, powerful, and modular. Mastering it is essential for building production-grade LLM applications.

###  Hands-on Steps You Will Perform
- Initialize Gemini via LangChain
- Build runnable sequences
- Test different runnable types
- Compose pipelines end-to-end


In [1]:
from secrete_key import my_gemini_api_key
API_KEY = my_gemini_api_key()


In [2]:
!pip install -qU langchain langchain-google-genai google-generativeai


'pip' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate

model = ChatGoogleGenerativeAI(
    model='gemini-2.0-flash',
    google_api_key=API_KEY
)
model


c:\Users\Selvam Sabarish\AppData\Local\Python\pythoncore-3.12-64\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ChatGoogleGenerativeAI(model='models/gemini-2.0-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000001EB547C5940>, default_metadata=())

#  1. What Is a Runnable?


`Runnable` is LangChains universal execution abstraction. **Everything becomes a runnable**:
- Prompts
- Models
- Chains
- Lambdas (functions)
- Maps (parallel execution)
- Sequences (pipeline execution)

This makes it incredibly easy to compose complex LLM workflows.


In [11]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnableSequence

prompt = PromptTemplate.from_template("Explain: {topic}")

chain = RunnableSequence(prompt,model)

chain

response = chain.invoke({"topic": "quantum computing"})

response.content


"Okay, let's break down quantum computing in a way that's understandable. It's a complex topic, but we can cover the key concepts.\n\n**What is Quantum Computing? (In a Nutshell)**\n\nQuantum computing is a new type of computation that leverages the principles of quantum mechanics to solve problems that are too complex for classical computers. Instead of using bits that are either 0 or 1, quantum computers use **qubits** that can be 0, 1, or *both* simultaneously. This allows them to explore many possibilities at once, potentially leading to exponential speedups for certain types of calculations.\n\n**Key Concepts Explained:**\n\n1. **Classical Bits vs. Quantum Bits (Qubits):**\n\n   *   **Classical Bits:**  The fundamental unit of information in a classical computer.  A bit can be either a 0 or a 1.  Think of it like a light switch that's either on or off.\n\n   *   **Qubits:** The fundamental unit of information in a quantum computer.  Unlike bits, qubits can exist in a superposition

#  2. Types of Runnables


## 2.1 `RunnableLambda`
A lightweight wrapper around a Python function.
Perfect for preprocessing, validation, or custom logic.


In [13]:
upper = RunnableLambda(lambda x: x.upper())
upper.invoke("hello runnable!")


'HELLO RUNNABLE!'

## 2.2 `RunnableMap`
Run multiple tasks in **parallel**  powerful for multi-extraction tasks.


In [14]:
from langchain_core.runnables import RunnableMap

process = RunnableMap({
    "summary": RunnableLambda(lambda x: x[:50]),
    "length": RunnableLambda(lambda x: len(x)),
})

process.invoke("LangChain makes LLM workflows modular and composable!")


{'summary': 'LangChain makes LLM workflows modular and composab', 'length': 53}

## 2.3 `RunnableSequence`
Create linear pipelines: **step  step  step**


In [18]:
sequence = RunnableSequence(
    RunnableLambda(lambda x: f"Processed: {x}"),
    RunnableLambda(lambda x: x + " "),
)

sequence.invoke("Welcome to Bulerez!")


'Processed: Welcome to Bulerez! ✔️'

## 2.4 `RunnableBranch`
Conditional branching logic  choose different pipelines based on input.


In [ ]:
from langchain_core.runnables import RunnableBranch

branch = RunnableBranch(
    (lambda x: "hello" in x.lower(), RunnableLambda(lambda x: "Greeting detected!")),
    RunnableLambda(lambda x: "No greeting found."),
)

branch.invoke("Hello there!")


'Greeting detected!'

#  3. Async vs Sync Execution

Runnables support two powerful execution modes  **Sync** and **Async**  enabling flexibility for both simple and high-performance AI workflows.


---

###  Sync vs Async in LangChain  Beginner-Friendly Explanation

LangChain Runnables can be executed in **two different ways**, depending on your needs:

* **Sync**  Traditional, simple, step-by-step execution
* **Async**  Non-blocking, parallel, faster multi-call execution

Lets break them down clearly.

---

### 1 Sync Mode  `.invoke()`

Sync = **You call  You wait  You get the output**.

```
You  chain.invoke()  wait  result
```

### When to Use Sync Mode

* Basic testing
* Running one LLM call at a time
* Simple scripts
* Learning or debugging

### Example

```python
result = chain.invoke({"topic": "quantum computing"})
print(result)
```

---

### 2 Async Mode  `.ainvoke()`

Async = **Non-blocking execution**.
Your program doesn't pause while waiting for the model response.

This means you can:

* Run **multiple LLM calls at the same time**
* Speed up heavy pipelines
* Build fast backend services

### Real-Life Analogy

Async is like ordering food at multiple counters simultaneously:

```
You  send many orders  each gets prepared in parallel  results return faster
```

---

###  Example: Async `.ainvoke()` (Beginner-Friendly)

```python
import asyncio

# Define an async function
async def demo():
    # 'ainvoke' runs the chain without blocking the program
    result = await chain.ainvoke({"topic": "async execution in LangChain"})
    print(result)

# Run the async function
await demo()
```

---
###  Whats Happening Here?

* **`async def demo():`**
  Creates an async function that can run asynchronous tasks.

* **`await chain.ainvoke({...})`**
  Calls the LLM *without blocking* your program.

* **`await demo()`**
  Executes the async function.

This allows your application to remain fast and responsive  even during multiple model calls.

---

###  Summary (Simple & Clear)

| Mode      | Method       | When to Use                    | Behavior                    |
| --------- | ------------ | ------------------------------ | --------------------------- |
| **Sync**  | `.invoke()`  | Simple or single tasks         | Blocking (waits for result) |
| **Async** | `.ainvoke()` | Many calls, speed, parallelism | Non-blocking (much faster)  |


In [31]:
import asyncio

async def demo():
    result = await chain.ainvoke({"topic": "async execution in LangChain"})
    print(result)

await demo()


content='Asynchronous execution in LangChain allows you to run tasks concurrently, improving the overall performance and efficiency of your applications.  It\'s particularly beneficial when dealing with operations that involve waiting, such as:\n\n*   **API calls:** Querying external APIs (like LLMs) often involves network latency.\n*   **File I/O:** Reading from or writing to files can take time.\n*   **Other I/O-bound operations:** Any task that spends a significant amount of time waiting for external resources.\n\nBy using asynchronous execution, your LangChain application can perform other tasks while waiting for these operations to complete, rather than blocking and idling.\n\nHere\'s a breakdown of how async execution works in LangChain and why it\'s useful:\n\n**Key Concepts**\n\n*   **`async` and `await`:** These are keywords in Python that enable asynchronous programming.  `async` declares a function as a coroutine, which can be paused and resumed.  `await` is used to pause th

#  4. Composability in Runnables
Build full pipelines:
`preprocess  prompt  model  postprocess`


In [30]:
pipeline = (
    RunnableLambda(lambda x: {"topic": x.strip()})
    | prompt
    | model
    | RunnableLambda(lambda x: x.content)
)

pipeline.invoke("   LangChain pipelines   ")


'LangChain pipelines, or chains, are a fundamental concept in the LangChain framework. They represent a sequence of calls to different components, linking them together to achieve a more complex task than any single component could accomplish on its own. Think of it like an assembly line where each station performs a specific operation, and the output of one station becomes the input of the next.\n\nHere\'s a breakdown of what they are and why they are important:\n\n**What are LangChain Chains?**\n\n* **Sequences of Calls:** At their core, a chain is a sequence of calls, typically to Large Language Models (LLMs), but also to other components like data connectors, document loaders, vector databases, and more.\n\n* **Connecting Components:**  Chains allow you to connect these individual components in a logical order, defining the flow of information and processing.\n\n* **Modular and Reusable:** Chains are designed to be modular, meaning you can combine different chains to create even mo

#  Summary
You now understand:
- What the Runnable interface is
- Runnable types: Map, Sequence, Lambda, Branch
- Sync vs async execution
- How to compose powerful pipelines

Youre now ready to build advanced LangChain workflows!
